# Amazon Bedrock 入門 Workshop 
# Chapter1: 基本的なモデル呼び出し

この章では Amazon Bedrock 上の基盤モデルを呼び出すための基本的な方法を体験していただきます。  
なお、実行環境は SageMaker Notebook を前提にしているため、他の環境（ローカル PC や SageMaker Studio Notebook）で実行する場合は適宜追加モジュールのインストールや IAM の権限設定が必要になる可能性があります。


## 目次

- [事前準備](#事前準備)
- [Python SDK での Amazon Bedrock の利用](#python-sdk-での-amazon-bedrock-の利用)
- [テキスト生成モデルの利用](#テキスト生成モデルの利用)
- [様々な基盤モデルの利用](#様々な基盤モデルの利用)
- [まとめ](#まとめ)

## 事前準備

###  Amazon Bedrock 上での基盤モデルの有効化

初めて Amazon Bedrock を使われる場合、モデル有効化手続きが必要となります。[ドキュメントの手順](https://docs.aws.amazon.com/ja_jp/bedrock/latest/userguide/model-access.html)を参考に有効化をお願いします。


### 必要権限の付与

強めの権限にはなりますが、本コンテンツを実行するのに必要な権限を付与するために、下記ポリシーを SageMaker Studio の実行ロールにアタッチします。  

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "Statement1",
            "Effect": "Allow",
            "Action": "bedrock:*",
            "Resource": "*"
        }
    ]
}
```

詳しくは[こちらのドキュメント](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html)を参照してください。


### ライブラリの準備
以下のセルを実行して必要な ```boto3, awscli, botocore, langchain``` パッケージをインストールします

In [ ]:
%pip install boto3 awscli botocore langchain

## Python SDK での Amazon Bedrock の利用

Amazon Bedrock を呼び出すための boto3 client を作成します。  
クライアントには、Bedrock Cllent, Bedrock Runtime Client, Bedrock Agent Client, Bedrock Agent Runtime Client の 4 種類が存在します。 
特に、基盤モデルを利用する際に用いるのは、Bedrock Cllent, Bedrock Runtime Client の 2 つになります。

[Bedrock Client](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock.html) が基盤モデル一覧の表示や、カスタマイズジョブの管理など、制御面を扱うクライアントになります。     

[Bedrock Runtime Client](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-runtime.html) は基盤モデルの呼び出しに利用するクライアントになります。

それでは、早速 Bedrock Client を使って利用可能な基盤モデルの一覧を取得します。

In [ ]:
import boto3
import json
# bedrock client
bedrock_client = boto3.client(service_name='bedrock')
bedrock_runtime_client = boto3.client(service_name='bedrock-runtime')

In [ ]:
# 利用可能な基盤モデル一覧の表示
response_json = bedrock_client.list_foundation_models()

for models in response_json['modelSummaries']:
    print(models['modelId'])

## テキスト生成モデルの利用

続いて、Bedrock Runtime Client を使って基盤モデルを呼び出し、文章を生成します。   
invoke_model関数を使うことで、基盤モデルを呼び出すことができます。   
以下では、 Anthropic Claude Instant を使った例を示します。

In [ ]:
# 基盤モデルを同期的に呼び出して推論する (Anthropic claude-instant-v1 の場合)

model_id = 'anthropic.claude-instant-v1'
system_prompt = "あなたは生成AIのエージェントです。ユーザからの質問に丁寧に回答してください。"
max_tokens = 1000
temperature = 0

# ユーザからの指示を記述する
user_message = {
        "role": "user",
        "content": "日本で一番高い山を教えてください"       
    }

body = json.dumps(
    {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": max_tokens,
        "system": system_prompt,
        "messages": [user_message],
        "temperature": temperature
    }  
)  

response = bedrock_runtime_client.invoke_model(body=body, modelId=model_id)
response_json = json.loads(response.get('body').read())

# 回答の表示
print('#回答')
print(response_json['content'][0]['text'])

また、基盤モデルはストリーム形式で呼び出すことも可能です

In [ ]:
# 基盤モデルの推論結果をストリーム形式で得る (Anthropic claude-instant-v1 の場合)

model_id = 'anthropic.claude-instant-v1'
system_prompt = "あなたは生成AIのエージェントです。ユーザからの質問に丁寧に回答してください。"
max_tokens = 1000
temperature = 0

# ユーザからの指示を記述する
user_message = {
        "role": "user",
        "content": "日本で一番高い山を教えてください"       
    }

body = json.dumps(
    {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": max_tokens,
        "system": system_prompt,
        "messages": [user_message],
        "temperature": temperature
    }  
)  

response = bedrock_runtime_client.invoke_model_with_response_stream(body=body, modelId=model_id)

# 結果を受け取った分から出力する   
for body in response.get('body'):
    response_json = json.loads(body['chunk']['bytes'].decode())

    if response_json['type'] == 'message_start': 
        print('#ストリーム開始')
        continue
    if response_json['type'] == 'message_stop': 
        print('\n#ストリーム終了')
        continue
    if response_json['type'] == 'content_block_start':
        continue
    if response_json['type'] == 'content_block_stop':
        continue
    if response_json['type'] == 'content_block_delta':
        print(response_json['delta']['text'], end='')



### 画像を入力としたテキスト生成

Claude 3モデルでは、画像を入力とした呼び出しにも対応しています。以下はマルチモーダル呼び出しを行うコードの例になります。  

In [ ]:
import base64

# 画像を読み込みbase64形式にエンコードする。
img_path = 'sample.png'

with open(img_path, "rb") as f:
    img_data = f.read()
    base64_data = base64.b64encode(img_data)
    base64_str = base64_data.decode("utf-8")

# モデルへのプロンプト
model_id = 'anthropic.claude-3-sonnet-20240229-v1:0'
system_prompt = "あなたは生成AIのエージェントです。ユーザからの質問に丁寧に回答してください。"
max_tokens = 1000
temperature = 0

# ユーザからの指示と画像の内容を記述する
user_message = {
    "role": "user",
    "content": [
        {
            "type": "image",
            "source": {
                "type": "base64",
                "media_type": "image/png",
                "data": base64_str
            }
        },
        {
            "type": "text",
            "text": "画像に写っているものを説明してください。"
        }
    ]
}

body = json.dumps(
    {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": max_tokens,
        "system": system_prompt,
        "messages": [user_message]
    }  
) 

try:
    # 基盤モデルの呼び出し
    response = bedrock_runtime_client.invoke_model(body=body, modelId=model_id).get('body').read()
    response_json = json.loads(response)

    # 回答の表示
    print('#回答')
    print(response_json['content'][0]['text'])

except ClientError as err:
    message=err.response["Error"]["Message"]
    logger.error("A client error occurred: %s", message)
    print("A client error occured: " +
        format(message))

次に、マルチモーダルモデルをストリーム形式で呼び出します。

In [ ]:
# 基盤モデルのマルチモーダルでの推論結果をストリーム形式で得る (Anthropic claude-instant-v1 の場合)

# 画像を読み込みbase64形式にエンコードする。
img_path = 'sample.png'

with open(img_path, "rb") as f:
    img_data = f.read()
    base64_data = base64.b64encode(img_data)
    base64_str = base64_data.decode("utf-8")

model_id = 'anthropic.claude-3-sonnet-20240229-v1:0'
system_prompt = "あなたは生成AIのエージェントです。ユーザからの質問に丁寧に回答してください。"
max_tokens = 1000
temperature = 0

# ユーザからの指示と画像を記述する
user_message = {
    "role": "user",
    "content": [
        {
            "type": "image",
            "source": {
                "type": "base64",
                "media_type": "image/png",
                "data": base64_str
            }
        },
        {
            "type": "text",
            "text": "画像に写っているものを説明してください。"
        }
    ]
}

body = json.dumps(
    {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": max_tokens,
        "system": system_prompt,
        "messages": [user_message],
        "temperature": temperature
    }  
)  

response = bedrock_runtime_client.invoke_model_with_response_stream(body=body, modelId=model_id)

# 結果を受け取った分から出力する   
for body in response.get('body'):
    response_json = json.loads(body['chunk']['bytes'].decode())

    if response_json['type'] == 'message_start': 
        print('#ストリーム開始')
        continue
    if response_json['type'] == 'message_stop': 
        print('\n#ストリーム終了')
        continue
    if response_json['type'] == 'content_block_start':
        continue
    if response_json['type'] == 'content_block_stop':
        continue
    if response_json['type'] == 'content_block_delta':
        print(response_json['delta']['text'], end='')

## 様々な基盤モデルの利用

Bedrock では、テキスト生成に限らず、画像生成や埋め込み表現生成が可能です。

2024/3/15 現在、オレゴン・バージニア北部リージョンにおいて以下のモデルが呼び出し可能です。  

テキスト生成

- AI21 Labs: Jurassic-2 Mid
- AI21 Labs: Jurassic-2 Ultra
- Amazon: Titan Text G1 - Lite
- Amazon: Titan Text G1 - Express
- Anthropic: Claude 2
- Anthropic: Claude 2.1
- Anthropic: Claude 3 Sonnet
- Anthropic: Claude 3 Haiku
- Anthropic: Claude Instant
- Cohere: Command
- Cohere: Command Light
- Meta: Llama 2 Chat 13B
- Meta: Llama 2 Chat 70B
- Mistral AI: Mistral 7B Instruct
- Mistral AI: Mixtral 8x7B Instruct

画像からのテキスト生成

- Anthropic: Claude 3 Sonnet
- Anthropic: Claude 3 Haiku

画像生成

- Amazon: Titan Image Generator G1
- Stability AI: StableDiffusion XL 1.0
- Stability AI: StableDiffusion XL 0.8 (Deprecated)

埋め込み表現生成

- Amazon: Titan Embeddings G1 - Text
- Amazon: Titan Multimodal Embeddings G1  
- Cohere: Embed English
- Cohere: Embed Multilingual


以下では、画像生成モデルや埋め込み表現生成モデルを使った実行例を示します。

### 画像生成モデルの呼び出し  
実行する際はコメントアウトを外してください(行頭と行末の`'''`を外してください)

In [ ]:
import base64
import io
from PIL import Image
from IPython.display import display
import botocore

In [ ]:
'''
# SDXLモデルは 2024/2/6 現在バージニア北部リージョンとオレゴンリージョンにのみ存在するため、別でクライアントを定義します
bedrock_sdxl_client = boto3.client(service_name='bedrock-runtime', region_name='us-east-1')

prompt_data = "a fine image of an astronaut riding a horse on Mars"
body = json.dumps({
    "text_prompts": [{"text": prompt_data}],
    "cfg_scale": 10,
    "seed": 20,
    "steps": 50
})
modelId = "stability.stable-diffusion-xl"
accept = "application/json"
contentType = "application/json"

try:
    
    response = bedrock_sdxl_client.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())
    
    print(response_body["result"])
    print(f'{response_body.get("artifacts")[0].get("base64")[0:80]}...')

except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

'''

出力結果は base64 でエンコードされているため、デコードしてから表示します。

In [ ]:
'''
base_64_img_str = response_body.get("artifacts")[0].get("base64")
image = Image.open(io.BytesIO(base64.decodebytes(bytes(base_64_img_str, "utf-8"))))
display(image)
'''

### 埋め込み表現の生成

Titan Embeddings により文字列の埋め込み表現を取得することができます。  
埋め込み表現とは、文字列の性質を表現したベクトルであり、検索やレコメンデーション等で利用可能です。

In [ ]:
import numpy as np
# 入力文字列の埋め込み表現を取得する関数
def get_embedding(text: str) -> np.ndarray:
    body = json.dumps({
    "inputText": text
    })

    modelId =  "amazon.titan-embed-text-v1"
    accept = '*/*'
    contentType = 'application/json'

    response = bedrock_runtime_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())
    # print(response)
    response_embeddings = np.array(response_body.get('embedding'))
    
    return response_embeddings

出力結果は1536次元のベクトルになります

In [ ]:
result = get_embedding('日本で一番高い山は？')
print(result[:20])
print('shape:', result.shape)

ベクトルの類似度を計算することで、2つの文章が意味合いとして似ている度合いを定量化することができます。  
ここでは、文書どうしの類似度計算によく使われる指標である、[コサイン類似度](https://atmarkit.itmedia.co.jp/ait/articles/2112/08/news020.html)を使って、2つの文章を比較します。  

In [ ]:
# 2つのベクトルのコサイン類似度を計算する関数
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [ ]:
print(cos_sim(get_embedding('日本で一番高い山は？'), get_embedding('日本最高峰とされるのは？')))

In [ ]:
print(cos_sim(get_embedding('日本で一番高い山は？'), get_embedding('バナナはおやつに入りますか？')))

類似する文章のコサイン類似度は高く、話題が異なる文章は低くなることがわかります。  

## まとめ

このノートブックでは、AWS Python SDK(boto3) や LangChain を用いて Amazon Bedrock のモデルを呼び出すための基本的な方法をご紹介しました。  
次の章からは、より実践的に活用するための方法をご紹介していきます。